# Alchemix Code Review
## Intro
This document will contain the main information on the unpicking of the alchemix code. Additional/ supporting documents can be put in the 'alchemixStudy' folder.
This will serve as a central reference of our understanding of the alchemix smart contracts.
## Key questions
While understanding the code, we should pay special attention to the following, as they will be useful in our project:
* How are deposits and withdrawals handled?
* How are Yearn yields tracked? - this will be key for us as we must monitor yields coming in from yearn, to be able to show users their renumeration for performing rebalancing actions.
* Security - which security features have been implemented? Which seem different to standard (perhaps whitelisting)? Would these be useful for us?
* Which contracts can we leverage? - contracts such as wEth gateway and yearn token adaptor should be useful to us and we can implement them directly in a diamond pattern. Are there any others?
### Yearn Yields
function _preemptivelyHarvest(line 1002 AlchemistV2.sol)  
![preemptively harvest definition](./IMG/_preemptiveHarvest.png "preemptively harvest")  
**current amount - expected amount** gives the number of underlying tokens which have been accrued in yield. (expected amount is updated by _sync) which is called at deposit, withdrawal and liquidation. This number of underlying tokens is converted to yield tokens by calling '_convertUnderlyingTokensToYield'. This amount of yield tokens are unwrapped, with the resulting underlying tokens being split between fees and user's credit. 
We should do something similar but need to think about timings/ order - i dont think we need to unwrap immediately - unwrap will occur on deposit/ withdrawal to pay rewards.
#### Vault tokens
The yearn vault tokens (e.g yvWeth), which are referred to as 'yieldTokens' in the alchemix docs, are like a deposit receipt. As the vault earns yield, the 'pricePerShare' of the vault tokens will increase. ie. a user holds a given number of yvtokens whcih does not change, however as the prootocol earns yield, the number of underlying tokens they are entitled to increases. Upon withdrawal of the underlying token, the yvtoken is burned. SImilarly, yvtokens are minted upon deposits of the underlying token to the vault. 

### YieldTokenParams
A struct defined in IAlchemistV2State.sol defining yield token parameters.
It is inherited by IAlchemistV2 which is inherited by AlchemistV2.sol.
It kind of acts as a bit of a glossary of terms:  
![yield token parameters](./IMG/yieldTokenParams.png "yield token parameters")

## Deposit function flow
There are two options for depositing to alchemix:
* depositUnderlying -'Deposit an underlying token into the account of `recipient` as `yieldToken`'.
* deposit - 'Deposit a yield token into a user's account.'
These functions are broken down in the next few sections.
### depositUnderlying
deposit underlying is a function on both the WETHGateway contract and AlchemistV2 contract. 
#### WETHGateway implementation 
When a user uses the website to deposit, a transaction calling 'depositUnderlying' on the WETHGateway contract is created. This has a modifier _onlyWhitelisted which is described below.  
WETHGateway then calls deposit on the WETH instance which is initialised in the constructor.  
Finally, depositUnderlying is called on the AlchemistV2 contract. 
##### _onlyWhitelisted()
OnlyWhitelisted is a function modifier which appears throughout the contracts.  
It first checks wether the message sender is an EOA (Externally Owned Account), controlled by public+private keys - basically a user wallet). If the caller is not an EOA, it must be a contract, and if it is not a whitelisted contract the transaction is reverted.
#### AlchemistV2 implementation
Checks:
* _onlyWhitelisted - see above
* _checkArgument(recipient != address(0)) - Checks the recipient is not the nulll address, reverting if it is
* _checkSupportedYieldToken(yieldToken) - checks the yield token is in the _supportedYieldTokens mapping.  

Calls:
* function calls _wrap (see below)
* function calls _deposit, depositing yield tokens to recipients account. The front end sets the recipient to the caller address.

_deposit is quite a beefy function, will break it down below.

##### _wrap
* loads yield token parameters from _yieldTokens mapping
* creates an instance of the adapter using the ITokenAdapter interface
* safe transfers underlying tokens to AlchemistV2 contract from user account.
* wraps tokens to 'wrapped shares' using adapter
* ensures wrapped shares amount > min amount out  

###### token adapter wrapping
Yearn token adapter's 'wrap' function does 3 things:
1. transfers underlying token from alchemist contract to token adaptor contract.
2. Approves underlying token to be spent by the yvweth token (yvweth is a token with added functionality, allowing use of strategies etc.)
3. calls 'deposit' on the yvweth token, which:
* mints yvweth
* sends yvweth to alchemist contract.  

There was some confusion regarding the yvweth tokens: two implementations were found:
* "0xa258C4606Ca8206D8aA700cE2143D7db854D168c" - labelled 'Alchemix Finance: Yearn yvWETH Vault' on etherscan but quoted at [yearn finance weth vault](https://yearn.finance/vaults/1/0xa258C4606Ca8206D8aA700cE2143D7db854D168c)
* "0xa9fE4601811213c340e850ea305481afF02f5b28" - labelled as yvweth on etehrscan. Yearn said there were two separate yvweth contracts, but it now seems only the 'alchemix' one is used.

###### Yearn vault token information
Upon wrapping, the yearn vault token contract (vault token) is given approval to spend the underlying tokens. This allows the yvweth token (a contract) to move the underlying token (Weth in this case) to strategies to earn yield.  
Inside the Yearn vault token contract there is a struct named strategy params for controlling the strategies and there is also an interface named 'Strategy' which will control the deployment of capital which the token (contract) has been given access to during wrapping.

## _deposit
Checks:
* Argument - that amount is > 0
* Yield and underlying tokens are enabled
* Token has not experienced a sudden loss using _checkLoss function - check how this works
* maximum expected value not breached

Does:  
* 'Buffers' any harvestable yield tokens using _preemptivelyHarvest
* _poke - 
* _issueSharesForAmount
* _sync - updates expected value. The last argument is a function, either _uadd or _usub which will add or subtract within the function depending which is passed in the calling function (deposit, withdraw, liquidate)
* 

## Sets
Sets are defined in contracts/libraries/sets.sol. Sets are a data structure which contain:
 * A list of addresses, named values
 * a mapping of addresses to uint256 named indexes.
 In AlchemistV2.sol, each account has an addressSet struct named deposited tokens. This keeps track of which tokens a user has deposited to the system.  
 
### Shares
The alchemist contract keeps a record of deposits as shares instead of specific units of various tokens.  
The number of shares for a depsoit is calculated using '_convertYieldTokensToShares'. It takes a yieldToken address and uint256 amount as arguments. This is called by the function '_issueSharesForAmount'.  
If there are originally no shares, the number of yield tokens is the number of shares.  
Otherwise, shares is calculated as: amount * _yieldTokens[yieldToken].totalShares / _calculateUnrealizedActiveBalance(yieldToken); (line 1611).  
_calculateUnrealizedActiveBalance  


## Resources/ References
[Alchemix contracts outline](https://alchemix-finance.gitbook.io/v2/)  
[Alchemix V2 Contracts Github](https://github.com/alchemix-finance/v2-contracts)  
[Alchemix user docs](https://alchemix-finance.gitbook.io/user-docs/)  
[OpenAI prompt design docs](https://beta.openai.com/docs/guides/completion/prompt-design)  
